In [1]:
# TODOs: Operate DB Class
# TODOs: Use SQLite
# TODOs: Implement BM25

from elastic_db import ElasticDB

# INIT DB OBJECT
PORT = "http://localhost:9200"
INDEX_NAME = "news_cc"

news_db = ElasticDB(elastic_port=PORT, elastic_index=INDEX_NAME)
wiki_db = ElasticDB(elastic_port=PORT, elastic_index="knowledge")

INFO:elastic_db:Connecting to http://localhost:9200 
INFO:elastic_db:Connected to <Elasticsearch(['http://localhost:9200'])> 
INFO:elastic_db:Connecting to http://localhost:9200 
INFO:elastic_db:Connected to <Elasticsearch(['http://localhost:9200'])> 


In [3]:
### LOAD DATA ###
import json

data = [json.loads(ln) for ln in open("../data/train_cmv_cleaned.jsonl")]
kp = [json.loads(ln) for ln in open("../data/keyphrases.jsonl")]
ex_retrieve = [json.loads(ln) for ln in open("../data/wiki_doc_retrieved_from_op_train.jsonlist")]
conan = [json.loads(ln) for ln in open("../data/CONAN.json")]

ex_ranked = [json.loads(ln) for ln in open("../data/selected_evidence.jsonl")]


In [4]:
### SAMPLE ###

sample_args = data[0:5]
sample_kp = kp[0:5]
sample_output = ex_retrieve[0:5]

sample_args[0]

{'id': 't3_2ro9ux',
 'titles': 'Anything that is manmade is natural.',
 'arguments': 'I cant remember the topic that spurred this discussion but a friend and I were debating whether manmade things were natural. He took the position that they are unnatural. He cited this definition by MerriamWebster existing in nature and not made or caused by people coming from nature as his basis for the distinction for natural vs. unnatural.However I respectfully disagree with his position and furthermore that definition of natural. People arise from nature. Humankinds capacity to create problemsolve analyze rationalize and build also come from natural processes. How are the things we create unnatural? It is only through natural occurrences that we have this ability why is it that we would give the credit of these things solely to man as opposed to nature? We are not separate from nature thus how can any of our actions or creations be unnatural? If we were somehow separate from nature I would underst

In [5]:
### SAMPLE OUTPUT ###

_ = 2
sample = sample_output[_]["retrieval_results"]
sample_ranked = ex_ranked[_]

# print(len(sample))
# sample

sample_ranked

{'cid': 'd98oiy7',
 'evidence_info': [{'sentence': 'At the time our country was founded major components of the economy were built on an immoral action that had been legal and normal for hundreds of years .',
   'evidence': ['Although this is often merely a rebuttable presumption, many jurisdictions have laws which restrict the possibility of legal rebuttal (for instance by creating a legal time limit during which paternity may be challengedsuch as a certain number of years from the birth of the child).',
    'In March 2012 the "Times of Swaziland" reported that a number of senators asked that the government take legal action against individuals who criticized King Mswati III on social networking sites.',
    'The Archbishop of Jos, Ignatius Ayau Kaigama, argued that the action was "in line with the moral and ethical values of the Nigerian and African cultures", and blessed President Goodluck Jonathan in not bowing to international pressure: "To protect you and yor administration again

In [12]:
import keyphrase_extraction
from keyphrase_extraction import exctract_keyphrase
import re

# TODOs: News Data
# TODOs: Ranking
# TODOs: Stance
# TODOs: Paralellise
# TODOs: Target Premises

ar_ = sample_args[_]["arguments"]
queries = []
sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', ar_)

from detection.BERT_adu_classifier import predict

premises = []
for sent in sentences:
    prediction = predict(sent)
    
    if prediction == "premise":
        premises.append((sent, prediction))

results = []
# Sentence Level
for i in sentences:
    
    # TODOs: Refactor key_BERT as Class for multiple oututs
    kp_ = exctract_keyphrase(i, n_gram=3, n_kp=3)
    
    query = ", ".join(i for i in kp_)
    res = [i["_source"]["document"]["title"] for i in news_db.search(query_=query, k=5)]

    results.append({
        "argument_sentence": i, 
        "query": query, 
        "retireved_documents": res,

    })

results

# results

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.336s]
INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.339s]
INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.202s]
INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.455s]
INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.381s]
INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.277s]
INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.465s]


[{'argument_sentence': 'Thinking about todays news BBC WikipediaI think France is in a tough spot in how to react to this.',
  'query': 'f, t, f',
  'retireved_documents': ['Reward offered in Center Point gun store burglary',
   'Banks lead S&P rally',
   'Jefferson Middle School Back-to-School 2017',
   'New Music: Tampa Tony ft. J. Towns - No Feelings',
   'Jeffrey Is Considering Re-Gifting a Christmas Gift… For Karen!']},
 {'argument_sentence': 'There would be even stronger calls for an antiislam and antiimmigration policies.',
  'query': 'a, c, a',
  'retireved_documents': ['NASCAR TV Schedule',
   'Tarps Now® Launches New Lines Of Military Grade Tarps And Fabrics',
   'Highway now clear from earlier crash',
   'Wales weather forecast: Met Office issues ice warning but warmer temperatures on the way',
   'Wales weather forecast: Met Office issues ice warning but warmer temperatures on the way']},
 {'argument_sentence': 'These event reminds people that they cannot depend on protecti

In [26]:
# Argument Level

results = []

_ = 90
hate_ = sample_hate[0]["conan"][_]["hateSpeech"]

# queries = []
# sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', hate_)

results = []

# TODOs: Refactor key_BERT as Class for multiple oututs
#kp_ = (exctract_keyphrase(hate_, n_gram=1, n_kp=1), exctract_keyphrase(hate_, n_gram=2, n_kp=1), exctract_keyphrase(hate_, n_gram=3, n_kp=1))

kp_ = (exctract_keyphrase(hate_, n_gram=5, n_kp=1))

query = " ,".join(i for i in kp_)
query

res = [i["_source"]["document"]["title"] for i in news_db.search(query_=query, k=5)]

results.append({
    "argument_sentence": hate_, 
    "query": query, 
    "retireved_documents": res,

})

results


INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.367s]


[{'argument_sentence': 'Fuck Islam and Fuck ISIS. We should repatriate all Muslims. They are all a bunch of violent criminals who do not share our values and never will. Without them Britain will be a better, safer place.',
  'query': 'fuck isis repatriate muslims',
  'retireved_documents': ['‘Assalamu Alaykom:’ peace be upon you',
   'How We Got to Here: A Charlottesville Reading List',
   'De-radicalization can work for former ISIS fighters',
   'How to Get Assyrians to Stay',
   'How to Get Christians to Stay']}]

In [27]:
results = []
res = [i["_source"]["document"]["title"] for i in wiki_db.search(query_=query, k=5)]

results.append({
    "argument_sentence": hate_, 
    "query": query, 
    "retireved_documents": res,

})

results

INFO:elastic_transport.transport:POST http://localhost:9200/knowledge/_search [status:200 duration:0.249s]


[{'argument_sentence': 'Fuck Islam and Fuck ISIS. We should repatriate all Muslims. They are all a bunch of violent criminals who do not share our values and never will. Without them Britain will be a better, safer place.',
  'query': 'fuck isis repatriate muslims',
  'retireved_documents': ['Refuting ISIS',
   'Refuting ISIS',
   'Take Off (2017 film)',
   'Terrorism in India',
   'Tareq Kamleh']}]

In [ ]:
#queries
query = "the executive the presidential system legislative branches partisan unproductive"
res = search_text(wiki_ev, query_=query, k=3)
res

[{'_index': 'knowledge',
  '_id': 'kbZa5YIBRqC1428DaQUk',
  '_score': 33.672318,
  '_ignored': ['document.text.keyword'],
  '_source': {'document': {'id': '3408791',
    'source': 'wikipedia',
    'title': 'Ruling party',
    'text': "In parliamentary systems, the majority in the legislature also controls the executive branch of government, thus leaving no possibility of opposing parties concurrently occupying the executive and legislative branches of government. In other systems, such as in an American style presidential system, the party of the president does not necessarily also have a legislative majority. A 'ruling party' is also used to describe the party of one-party states, such as the Chinese Communist Party in the People's Republic of China. In his political manifesto 'The Green Book', the late Libyan leader Muammar al-Gaddafi attacked the ability of the ruling party, using it as a basis for his opposition to partisan politics. None"}}},
 {'_index': 'knowledge',
  '_id': 'JQ3

In [ ]:
import re

print(", ".join(re.sub(r'\[\[(?:[^|\]]*\|)?([^\]]*)]]', "", i["_source"]["document"]["title"]).strip("[]") for i in res))

Ruling party, Fusion of powers, Student governments in the United States


In [ ]:
### TEST SEARCH ###

def search_text(es, query_, k=5):
    results = es.search(
        index = es.elastic_index,
        query = {
            "match": {
                "document.text": query_,
                },
        },
        size=k)

    hits = results["hits"]["hits"]
    doc_ids = [row['_source']["document"]["id"] for row in hits]

    return hits

def search_topic(es, topic, k=5):
    results = es.search(
        index = es.elastic_index,
        body= {
            "size": k,
            "query": {
                "match": {
                    "document.title": topic,
        }}})

    hits = results["hits"]["hits"]
    doc_ids = [row['_source']["document"]["id"] for row in hits]

    title = hits[0]["_source"]["document"]["title"]
    text = hits[0]["_source"]["document"]["text"]

    return {
        "title": title,
        "text": text
    }

query = "government emails"
text = search_text(wiki_ev, query_=query, k=10)

print(text)


In [ ]:
### SEARCH FUNCTION ###

# TODOs: Implement as Class-DB object

# def search_text(db, query_, k=5):
#     results = db.search(
#         index = es.elastic_index,
#         query = {
#             "match": {
#                 "document.text": query_,
#                 },
#         },
#         size=k)

#     hits = results["hits"]["hits"]
#     doc_ids = [row['_source']["document"]["id"] for row in hits]

#     return hits